In [1]:
import logging
import os
import pandas as pd
import json

from pathlib import Path

from copy import copy

%load_ext autoreload
%autoreload 2

# Disable logging of the program in the notebook
os.environ["LOGLEVEL"] = "CRITICAL"

logging.basicConfig()
logging.root.setLevel(logging.INFO)
logging.basicConfig(level=logging.INFO)

logger = logging.getLogger('__main__')
logger.setLevel(os.environ.get("LOGLEVEL", logging.FATAL))

pybooklogger = logging.getLogger('pybook')
pybooklogger.setLevel(logging.DEBUG)

%aimport setup

%load_ext autoreload
%autoreload 2

from setup import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Track Layout
Calculate the layout of the dutch railway system


In [2]:
layout_file =   "../data/prorail/parsed/netherlands-schiphol.json"
layout = Layout(layout_file)

KeyboardInterrupt: 

In [ ]:
edges_df = pd.DataFrame({"Outgoing routes": [len(n.outgoing) for n in layout.g_block.nodes.values() if len(n.outgoing) <= 25]})
hist = edges_df.hist(bins=25, )
plt.xlabel("Number of outgoing routes")
plt.ylabel("Number of occurrences")
plt.show()

# Experiment Runtime Agents


In [ ]:
scenario_file = "../data/prorail/scenarios/RT/2025-07-04_1.json"
exp_settings = {
    "origin": "RTD|9", 
    "destination": "ASDZ|2",
    "start_time": 900,
    "max_buffer_time": 900,
    "use_recovery_time": True,
}
set_default(exp_settings)
experiments = []
try:
    base_path = Path(__file__).parent
    file_path = (base_path / scenario_file).resolve()
    data = json.load(open(file_path))
except:
    data = json.load(open(scenario_file))
pybooklogger.setLevel(logging.INFO)
logger.setLevel(logging.CRITICAL)
for idx, entry in enumerate(data["trains"]):
    scenario_data = copy(data)
    scenario_data["trains"] = data["trains"][:idx+1]
    with open("temp_scenario.json", "w") as outfile:
        json.dump(scenario_data, outfile)
    scenario = Scenario(layout, "temp_scenario.json", -1)
    origin = exp_settings["origin"]
    destination = exp_settings["destination"]
    velocity = exp_settings["velocity"]
    start_time = exp_settings["start_time"]
    max_buffer_time = exp_settings["max_buffer_time"]
    use_recovery_time = exp_settings["use_recovery_time"]
    metadata = exp_settings["metadata"]
    metadata["label"] = f"Num trains: {idx + 1}"

    agent_id = scenario.agent_id
    pybooklogger.info(f"Setting up experiment {exp_settings}")

    origin_signal = scenario.l.station_to_block(origin, direction=1)
    destination_signal = scenario.l.station_to_block(destination, direction=1)
    agent = Agent(agent_id, origin_signal, destination_signal, velocity, start_time)

    experiments.append(Experiment(scenario, agent, max_buffer_time, use_recovery_time, metadata))


### Blocking staircase diagram
Showing the route of the agent with the most stops, its quite long.


In [ ]:
exp = experiments[-1]
exp.s.plot(-1, exp.buffer_times, exp.recovery_times, False)

In [ ]:
timeout = 7200
run_experiments(experiments, timeout)

## Results


### Time statistics

In [ ]:
def sum_cols(df1, cols, name):
    df2 = df1.drop(columns=cols)
    df2[name] = df1[cols].sum(axis=1)
    return df2

time_df = pd.DataFrame([exp.get_running_time() for exp in experiments], index=[exp.metadata['label'] for exp in experiments])

setup_cols = ["track graph creation", "routing graph creation"]
recompute_cols = ["unsafe interval generation", "safe interval generation", "bt and crt generation", "converting routes to blocks"]
search_cols = ["FlexSIPP search time"]

time_df = sum_cols(time_df, setup_cols, "Setup Time")
time_df = sum_cols(time_df, recompute_cols, "Recompute Time")
time_df = sum_cols(time_df, search_cols, "Search Time")
time_df

### Search Node Statistics


In [ ]:
nodes_df = pd.DataFrame([exp.get_complexity() for exp in experiments], index=[exp.metadata['label'] for exp in experiments])

nodes_df

### Output paths found


In [ ]:
for key, value in experiments[2].results[3].items():
    delayed_trains = {i: v for i,v in enumerate(value[0][4]) if float(v[0]) > 0}
    print(f"{key.replace('r-', '')}\nearliest departure: {int(min(float(value[0][1]), float(value[0][2])) / 60)}\ndepart before: {int(float(value[0][2]) / 60)}\narrive at: {int((float(value[0][1]) + float(value[0][3])) / 60)}\ndelays trains: {delayed_trains}")

### Path statistics


In [ ]:
for exp in experiments:
    print(f"Differend paths found for {exp.metadata['label']}: {sum(exp.results[2].values())}")